In [1]:
import requests
import json
import os
import pandas as pd
import random  # Added for random sampling

def get_file_names(folder_path):
    """Gets a list of file names in the specified folder."""
    file_names = []
    for entry in os.scandir(folder_path):
        if entry.is_file():
            file_names.append(entry.name)
    return file_names

def test_api():
    # Test health check
    try:
        response = requests.get("http://localhost:8008/ping")
        print("Health check response:", response.json())
    except Exception as e:
        print(f"Health check failed: {str(e)}")
        return

    # Initialize counters
    total_predictions = 0
    incorrect_predictions = 0
    errors = 0

    try:
        train_files = get_file_names("./../ML/train/")
        train_pd = pd.read_csv("./../ML/train.csv")

        # Get all malignant cases and sample 20 randomly
        malignant_cases = train_pd[train_pd['target'] == 1]
        sampled_cases = malignant_cases.sample(n=20, random_state=42)  # random_state for reproducibility

        for _, row in sampled_cases.iterrows():
            file_name = row['image_name']
            metadata = {
                'sex': row['sex'],
                'anatom_site_general_challenge': row['anatom_site_general_challenge'],
                'age_approx': float(row['age_approx'])
            }
            
            dicom_file = f"./../ML/train/{file_name}.dcm"
            try:
                with open(dicom_file, 'rb') as f:
                    files = {
                        'file': (dicom_file, f, 'application/dicom'),
                        'metadata': (None, json.dumps(metadata))
                    }
                    
                    response = requests.post("http://localhost:8008/predict", files=files)
                    total_predictions += 1
                    
                    if response.status_code == 200:
                        prediction = response.json()
                        # If prediction is 0 (meaning model predicted benign when it's malignant)
                        if prediction.get('prediction', 1) == 0:
                            incorrect_predictions += 1
                            print(f"\nIncorrect prediction for {file_name}:")
                            print(f"True label: 1, Predicted: {prediction.get('prediction')}")
                            print(f"Metadata: {metadata}")
                    else:
                        errors += 1
                        print(f"\nError for {file_name} (status code {response.status_code}):")
                        print(json.dumps(response.json(), indent=2))
            
            except Exception as e:
                errors += 1
                print(f"Error processing {file_name}: {str(e)}")

        # Print summary statistics
        print("\nPrediction Summary:")
        print(f"Total predictions made: {total_predictions}")
        print(f"Incorrect predictions: {incorrect_predictions}")
        print(f"Error rate: {(incorrect_predictions/total_predictions)*100:.2f}% ({incorrect_predictions}/{total_predictions})")
        print(f"Processing errors: {errors}")

    except Exception as e:
        print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    test_api()

Health check response: {'ping': 'pong!'}
Error processing ISIC_6767569: [Errno 2] No such file or directory: './../ML/train/ISIC_6767569.dcm'
Error processing ISIC_7312977: [Errno 2] No such file or directory: './../ML/train/ISIC_7312977.dcm'
Error processing ISIC_1785627: [Errno 2] No such file or directory: './../ML/train/ISIC_1785627.dcm'
Error processing ISIC_8066110: [Errno 2] No such file or directory: './../ML/train/ISIC_8066110.dcm'
Error processing ISIC_8838753: [Errno 2] No such file or directory: './../ML/train/ISIC_8838753.dcm'
Error processing ISIC_5369000: [Errno 2] No such file or directory: './../ML/train/ISIC_5369000.dcm'
Error processing ISIC_2153401: [Errno 2] No such file or directory: './../ML/train/ISIC_2153401.dcm'
Error processing ISIC_4584265: [Errno 2] No such file or directory: './../ML/train/ISIC_4584265.dcm'
Error processing ISIC_2408815: [Errno 2] No such file or directory: './../ML/train/ISIC_2408815.dcm'
Error processing ISIC_6496251: [Errno 2] No such f

In [1]:
import requests
import json
import os
import pandas as pd

def get_file_names(folder_path):
    """Gets a list of file names in the specified folder."""
    file_names = []
    for entry in os.scandir(folder_path):
        if entry.is_file():
            file_names.append(entry.name)
    return file_names

def test_api():
    # Test health check
    try:
        response = requests.get("http://localhost:8008/ping")
        print("Health check response:", response.json())
    except Exception as e:
        print(f"Health check failed: {str(e)}")
        return

    # Initialize counters
    total_predictions = 0
    incorrect_predictions = 0
    errors = 0

    try:
        train_files = get_file_names("./../ML/train/")
        train_pd = pd.read_csv("./../ML/train.csv")

        for i in train_files:
            file_name = str(i[:-4])
            matching_rows = train_pd.loc[(train_pd['image_name'] == file_name) & (train_pd['target'] == 0)]
            
            if not matching_rows.empty:
                metadata = {
                    'sex': matching_rows.sex.iloc[0],
                    'anatom_site_general_challenge': matching_rows.anatom_site_general_challenge.iloc[0],
                    'age_approx': float(matching_rows.age_approx.iloc[0])
                }
                
                dicom_file = f"./../ML/train/{file_name}.dcm"
                try:
                    with open(dicom_file, 'rb') as f:
                        files = {
                            'file': (dicom_file, f, 'application/dicom'),
                            'metadata': (None, json.dumps(metadata))
                        }
                        
                        response = requests.post("http://localhost:8008/predict", files=files)
                        total_predictions += 1
                        
                        if response.status_code == 200:
                            prediction = response.json()
                            # If prediction is not 0 (meaning model predicted malignant when it's benign)
                            if prediction.get('prediction', 0) != 0:
                                incorrect_predictions += 1
                                print(f"\nIncorrect prediction for {file_name}:")
                                print(f"True label: 0, Predicted: {prediction.get('prediction')}")
                                print(f"Metadata: {metadata}")
                        else:
                            errors += 1
                            print(f"\nError for {file_name} (status code {response.status_code}):")
                            print(json.dumps(response.json(), indent=2))
                
                except Exception as e:
                    errors += 1
                    print(f"Error processing {file_name}: {str(e)}")

        # Print summary statistics
        print("\nPrediction Summary:")
        print(f"Total predictions made: {total_predictions}")
        print(f"Incorrect predictions: {incorrect_predictions}")
        print(f"Error rate: {(incorrect_predictions/total_predictions)*100:.2f}% ({incorrect_predictions}/{total_predictions})")
        print(f"Processing errors: {errors}")

    except Exception as e:
        print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    test_api()

Health check response: {'ping': 'pong!'}

Incorrect prediction for ISIC_2452083:
True label: 0, Predicted: 1.0
Metadata: {'sex': 'female', 'anatom_site_general_challenge': 'upper extremity', 'age_approx': 20.0}

Incorrect prediction for ISIC_5624660:
True label: 0, Predicted: 1.0
Metadata: {'sex': 'male', 'anatom_site_general_challenge': 'head/neck', 'age_approx': 45.0}

Incorrect prediction for ISIC_7189099:
True label: 0, Predicted: 1.0
Metadata: {'sex': 'female', 'anatom_site_general_challenge': 'upper extremity', 'age_approx': 60.0}

Incorrect prediction for ISIC_5919794:
True label: 0, Predicted: 1.0
Metadata: {'sex': 'male', 'anatom_site_general_challenge': 'lower extremity', 'age_approx': 40.0}

Incorrect prediction for ISIC_1629539:
True label: 0, Predicted: 1.0
Metadata: {'sex': 'male', 'anatom_site_general_challenge': 'torso', 'age_approx': 60.0}

Incorrect prediction for ISIC_2131585:
True label: 0, Predicted: 1.0
Metadata: {'sex': 'female', 'anatom_site_general_challenge': 